use a neural net to pull out the horizon from an image and fit parameters to it to find the appropriate locations

In [1]:
import numpy as np
import matplotlib.pylab as plt
from eigsep_terrain.marjum_dem import MarjumDEM as DEM
import eigsep_terrain as et
import eigsep_terrain.utils as etu
import tqdm
import healpy
from matplotlib.image import imread
from matplotlib import transforms
from image_class import Image

In [2]:
# load image data
img0 = Image(pathname='/Users/komalkaur/Desktop/eigsep_stuff/hrzn_mapping/crosshair-imgs/img0.PNG', 
            label='img0', lat=39.247880, lon=-113.402623, alt=1698.0, roll=6.5, heading=88,
            angle_up=84.6, ver_weight=1/4, hor_weight=1/16, dis_weight=23*47)

img1 = Image(pathname='/Users/komalkaur/Desktop/eigsep_stuff/hrzn_mapping/crosshair-imgs/img1.PNG', 
            label='img1', lat=39.247441, lon=-113.402693, alt=1659.0, roll=3.6, heading=358,
            angle_up=49.0, ver_weight=1/24, hor_weight=1/8, dis_weight=13*14)

img2 = Image(pathname='/Users/komalkaur/Desktop/eigsep_stuff/hrzn_mapping/crosshair-imgs/img2.PNG', 
            label='img2', lat=39.247484, lon=-113.402730, alt=1695.0, roll=1.2, heading=352,
            angle_up=50, ver_weight=1/7, hor_weight=1/22, dis_weight=12*16)

img3 = Image(pathname='/Users/komalkaur/Desktop/eigsep_stuff/hrzn_mapping/crosshair-imgs/img3.PNG', 
            label='img3', lat=39.247797, lon=-113.402905, alt=1713.0, roll=30.3, heading=166,
            angle_up=84.8, ver_weight=1/32, hor_weight=1/6, dis_weight=11*10)

img4 = Image(pathname='/Users/komalkaur/Desktop/eigsep_stuff/hrzn_mapping/crosshair-imgs/img4.PNG', 
            label='img4', lat=39.247566, lon=-113.402991, alt=1746.0, roll=31.5, heading=150,
            angle_up=85.0, ver_weight=1/11, hor_weight=1/57, dis_weight=9*13)

img5 = Image(pathname='/Users/komalkaur/Desktop/eigsep_stuff/hrzn_mapping/crosshair-imgs/img5.PNG', 
            label='img5', lat=39.248385, lon=-113.401491, alt=1761.0, roll=7.0, heading=236,
            angle_up=13.0, ver_weight=1/7, hor_weight=1/5, dis_weight=4*6)

img6 = Image(pathname='/Users/komalkaur/Desktop/eigsep_stuff/hrzn_mapping/crosshair-imgs/img6.PNG', 
            label='img6', lat=39.248631, lon=-113.404227, alt=1834.0, roll=-6.5, heading=133,
            angle_up=-17.3, ver_weight=1/15, hor_weight=1/9, dis_weight=9*10)

img7 = Image(pathname='/Users/komalkaur/Desktop/eigsep_stuff/hrzn_mapping/crosshair-imgs/img7.PNG', 
            label='img7', lat=39.248400, lon=-113.401572, alt=1759.0, roll=0.8, heading=240,
            angle_up=17.4, ver_weight=1/18, hor_weight=1/8, dis_weight=6*7)


In [6]:
CACHE_FILE = 'marjum_dem.npz'
dem = DEM(cache_file=CACHE_FILE)


In [7]:
enu0 = dem.latlon_to_enu(img0.lat, img0.lon, img0.alt)
enu1 = dem.latlon_to_enu(img1.lat, img1.lon, img1.alt)
enu2 = dem.latlon_to_enu(img2.lat, img2.lon, img2.alt)
enu3 = dem.latlon_to_enu(img3.lat, img3.lon, img3.alt)
enu4 = dem.latlon_to_enu(img4.lat, img4.lon, img4.alt)
enu5 = dem.latlon_to_enu(img5.lat, img5.lon, img5.alt)
enu6 = dem.latlon_to_enu(img6.lat, img6.lon, img6.alt)
enu7 = dem.latlon_to_enu(img7.lat, img7.lon, img7.alt)

In [ ]:
angle_per_pix = 0.0007639913301538154   # rad per pix from mapping.ipynb

steps

1. fix the rotation of the photo
2. pull out the horizon shape
3. parameter fit to see which combination of photo enus would result in a matching horizon_pnts

In [12]:
def plot_img(ax, data, res=dem.res, xlabel=True, ylabel=True,
             colorbar=False, cmap='terrain', erng=None, nrng=None, **kw):
    '''Plot maps with standard format.'''
    if nrng is None:
        nrng = (0, data.shape[0] * res)
    if erng is None:
        erng = (0, data.shape[1] * res)
    extent = erng + nrng
    im = ax.imshow(data, extent=extent, cmap=cmap, origin='lower', interpolation='nearest', **kw)
    if colorbar:
        plt.colorbar(im)
    if xlabel:
        ax.set_xlabel('East [m]')
    if ylabel:
        ax.set_ylabel('North [m]')

def enu_hrzn_angle_plotter(east, north, up):
    '''plot horizon angles for a given enu idx'''
    # calculate horizon angles
    horizon_angles = []
    horizon_pnts = []
    pnt = np.array([east, north, up])
    delta_h = 0 #m
    _pnt = pnt + np.array([0, 0, delta_h])
    horizon_angles, horizon_pnts = dem.calc_horizon(*_pnt, n_az=512)

    # plot horizon angles
    fig, ax = plt.subplots()
    plot_img(ax, dem.data, dem.res)
    # for k in horizon_angles: # type: ignore
        # color = ax._get_lines.get_next_color()
    plt.plot(horizon_pnts[1], horizon_pnts[0], '.-') # type: ignore
    plt.plot(east, north, '+')
    _ = plt.title(f'Horizon Points for east: {np.round(east)}, north: {np.round(north)}, up: {np.round(up)}')

    return horizon_pnts